# Setup

## Google Only

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append('/content/drive/My Drive/Code/autocomplete_me')

In [23]:
!ls -ls '/content/drive/My Drive/Code/autocomplete_me'

total 17
4 drwx------ 3 root root 4096 Jul 11 13:00 data
4 drwx------ 2 root root 4096 Jul 11 13:26 models
4 drwx------ 2 root root 4096 Jul 11 13:01 notebooks
1 -rw------- 1 root root   57 Jul 11 08:44 requirements.txt
4 drwx------ 3 root root 4096 Jul 11 13:00 src


In [24]:
from src import utils, reader, predict_utils

In [5]:
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)

<module 'src.predict_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py'>

# Build Model

In [25]:
text = reader.read_bbc_tech()
content_type = 'BBC-tech'

In [10]:
text[0]

'US duo in first spam conviction\n\nA brother and sister in the US have been convicted of sending hundreds of thousands of unsolicited e-mail messages to AOL subscribers.\n\nIt is the first criminal prosecution of internet spam distributors. Jurors in Virginia recommended that the man, Jeremy Jaynes, serve nine years in prison and that his sister, Jessica DeGroot, be fined $7,500. They were convicted under a state law that bars the sending of bulk e-mails using fake addresses.\n\nThey will be formally sentenced next year. A third defendant, Richard Rutkowski, was acquitted. Prosecutors said Jaynes was "a snake oil salesman in a new format", using the internet to peddle useless wares, news agency Associated Press reported. A "Fed-Ex refund processor" was supposed to allow people to earn $75 an hour working from home. Another item on sale was an "internet history eraser". His sister helped him process credit card payments. Jaynes amassed a fortune of $24m from his sales, prosecutors said

## Training

### Long Text

In [11]:
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text)

There are 15491 unique words.
There are 199462 sequences.


In [12]:
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 5692 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:180: RuntimeWarning: invalid value encountered in true_divide
  embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.06223089,  0.03835242,  0.08488411, ..., -0.04284497,
         0.08662399, -0.00527513],
       ...,
       [-0.00346489, -0.08874972, -0.20891733, ...,  0.09497839,
        -0.12697276, -0.08340222],
       [-0.02349926, -0.06641477, -0.01852101, ...,  0.07821092,
         0.12694902,  0.1010958 ],
       [ 0.02721107,  0.00886596,  0.24643607, ...,  0.02826212,
         0.23311737,  0.06475616]])

#### Unidirectional, 1 layer, Untrainable

In [13]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=False,
    lstm_layers=1,
    bi_direc=False
)

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1549100   
_________________________________________________________________
masking_1 (Masking)          (None, None, 100)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 15491)             1998339   
Total params: 3,597,999
Trainable params: 2,048,899
Non-trainable params: 1,549,100
____________________________________

In [91]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-untrainable-50_seq',
    model=model,
    epochs=50
)

Train on 139623 samples, validate on 59839 samples
Epoch 1/50
139623/139623 [==============================] - 24s 172us/step - loss: 6.8772 - accuracy: 0.0524 - val_loss: 6.9880 - val_accuracy: 0.0685
Epoch 2/50
139623/139623 [==============================] - 24s 173us/step - loss: 6.8640 - accuracy: 0.0570 - val_loss: 6.9814 - val_accuracy: 0.0662
Epoch 3/50
139623/139623 [==============================] - 24s 173us/step - loss: 6.8464 - accuracy: 0.0600 - val_loss: 6.9647 - val_accuracy: 0.0665
Epoch 4/50
139623/139623 [==============================] - 24s 173us/step - loss: 6.8233 - accuracy: 0.0627 - val_loss: 6.9436 - val_accuracy: 0.0668
Epoch 5/50
139623/139623 [==============================] - 24s 172us/step - loss: 6.8005 - accuracy: 0.0637 - val_loss: 6.9284 - val_accuracy: 0.0667
Epoch 6/50
139623/139623 [==============================] - 24s 175us/step - loss: 6.7799 - accuracy: 0.0639 - val_loss: 6.9134 - val_accuracy: 0.0670
Epoch 7/50
139623/139623 [=================

#### Unidirectional, 2 layers, Untrainable

In [15]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=False,
    lstm_layers=2,
    bi_direc=False
)

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         1549100   
_________________________________________________________________
masking_2 (Masking)          (None, None, 100)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 15491)            

In [93]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-2_layers-untrainable-50_seq',
    model=model,
    epochs=50
)

Train on 139623 samples, validate on 59839 samples
Epoch 1/50
139623/139623 [==============================] - 40s 285us/step - loss: 7.7878 - accuracy: 0.0351 - val_loss: 6.9863 - val_accuracy: 0.0444
Epoch 2/50
139623/139623 [==============================] - 38s 275us/step - loss: 6.9763 - accuracy: 0.0438 - val_loss: 6.9976 - val_accuracy: 0.0451
Epoch 3/50
139623/139623 [==============================] - 39s 276us/step - loss: 6.9552 - accuracy: 0.0441 - val_loss: 7.0092 - val_accuracy: 0.0451
Epoch 4/50
139623/139623 [==============================] - 38s 274us/step - loss: 6.9405 - accuracy: 0.0427 - val_loss: 7.0090 - val_accuracy: 0.0451
Epoch 5/50
139623/139623 [==============================] - 38s 274us/step - loss: 6.9198 - accuracy: 0.0442 - val_loss: 6.9956 - val_accuracy: 0.0451
Epoch 6/50
139623/139623 [==============================] - 38s 275us/step - loss: 6.9020 - accuracy: 0.0457 - val_loss: 6.9958 - val_accuracy: 0.0459
Epoch 7/50
139623/139623 [=================

#### Bidirectional, 1 layer, Untrainable

In [17]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=False,
    lstm_layers=1,
    bi_direc=True
)

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         1549100   
_________________________________________________________________
masking_3 (Masking)          (None, None, 100)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 15491)             1998339   
Total params: 3,648,431
Trainable params: 2,099,331
Non-trainable params: 1,549,100
____________________________________

In [95]:
history = utils.train_model(
    training_dict,
    f'{content_type}_bi-1_layer-untrainable-50_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

Train on 139623 samples, validate on 59839 samples
Epoch 1/50
139623/139623 [==============================] - 39s 277us/step - loss: 7.7266 - accuracy: 0.0344 - val_loss: 6.9904 - val_accuracy: 0.0451
Epoch 2/50
139623/139623 [==============================] - 38s 270us/step - loss: 6.9665 - accuracy: 0.0436 - val_loss: 6.9881 - val_accuracy: 0.0451
Epoch 3/50
139623/139623 [==============================] - 38s 269us/step - loss: 6.9329 - accuracy: 0.0447 - val_loss: 6.9792 - val_accuracy: 0.0454
Epoch 4/50
139623/139623 [==============================] - 38s 269us/step - loss: 6.9037 - accuracy: 0.0465 - val_loss: 6.9731 - val_accuracy: 0.0435
Epoch 5/50
139623/139623 [==============================] - 38s 269us/step - loss: 6.8836 - accuracy: 0.0526 - val_loss: 6.9708 - val_accuracy: 0.0680
Epoch 6/50
139623/139623 [==============================] - 38s 270us/step - loss: 6.8647 - accuracy: 0.0590 - val_loss: 6.9547 - val_accuracy: 0.0681
Epoch 7/50
139623/139623 [=================

#### Bidirectional, 2 layers, Untrainable

In [96]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=False,
    lstm_layers=2,
    bi_direc=True
)

In [97]:
history = utils.train_model(
    training_dict,
    f'{content_type}_bi-2_layer-untrainable-50_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

Train on 139623 samples, validate on 59839 samples
Epoch 1/50
139623/139623 [==============================] - 53s 382us/step - loss: 7.7296 - accuracy: 0.0354 - val_loss: 6.9935 - val_accuracy: 0.0451
Epoch 2/50
139623/139623 [==============================] - 52s 371us/step - loss: 6.9771 - accuracy: 0.0425 - val_loss: 6.9994 - val_accuracy: 0.0451
Epoch 3/50
139623/139623 [==============================] - 52s 369us/step - loss: 6.9556 - accuracy: 0.0439 - val_loss: 7.0090 - val_accuracy: 0.0444
Epoch 4/50
139623/139623 [==============================] - 52s 373us/step - loss: 6.9372 - accuracy: 0.0439 - val_loss: 7.0026 - val_accuracy: 0.0443
Epoch 5/50
139623/139623 [==============================] - 52s 371us/step - loss: 6.9119 - accuracy: 0.0451 - val_loss: 6.9883 - val_accuracy: 0.0670
Epoch 6/50
139623/139623 [==============================] - 53s 383us/step - loss: 6.8953 - accuracy: 0.0482 - val_loss: 6.9871 - val_accuracy: 0.0682
Epoch 7/50
139623/139623 [=================

#### Unidirectional, 1 layer, Trainable

In [98]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False
)

In [99]:
train_history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-50_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 139623 samples, validate on 59839 samples
Epoch 1/50
139623/139623 [==============================] - 22s 155us/step - loss: 7.8261 - accuracy: 0.0328 - val_loss: 6.9788 - val_accuracy: 0.0451
Epoch 2/50
139623/139623 [==============================] - 21s 151us/step - loss: 6.9246 - accuracy: 0.0446 - val_loss: 6.9024 - val_accuracy: 0.0420
Epoch 3/50
139623/139623 [==============================] - 21s 151us/step - loss: 6.8223 - accuracy: 0.0436 - val_loss: 6.8884 - val_accuracy: 0.0444
Epoch 4/50
139623/139623 [==============================] - 21s 152us/step - loss: 6.7753 - accuracy: 0.0471 - val_loss: 6.8863 - val_accuracy: 0.0603
Epoch 5/50
139623/139623 [==============================] - 21s 151us/step - loss: 6.7243 - accuracy: 0.0608 - val_loss: 6.8378 - val_accuracy: 0.0687
Epoch 6/50
139623/139623 [==============================] - 21s 152us/step - loss: 6.6370 - accuracy: 0.0701 - val_loss: 6.7676 - val_accuracy: 0.0725
Epoch 7/50
139623/139623 [=================

In [ ]:
train_history

#### Bidirectional, 1 layer, Trainable

In [100]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=True
)

In [101]:
history = utils.train_model(
    training_dict,
    f'{content_type}_bi-1_layer-trainable-50_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 139623 samples, validate on 59839 samples
Epoch 1/50
139623/139623 [==============================] - 34s 241us/step - loss: 7.6994 - accuracy: 0.0339 - val_loss: 6.9884 - val_accuracy: 0.0451
Epoch 2/50
139623/139623 [==============================] - 33s 238us/step - loss: 6.9442 - accuracy: 0.0447 - val_loss: 6.9337 - val_accuracy: 0.0453
Epoch 3/50
139623/139623 [==============================] - 33s 239us/step - loss: 6.8310 - accuracy: 0.0469 - val_loss: 6.8793 - val_accuracy: 0.0659
Epoch 4/50
139623/139623 [==============================] - 33s 238us/step - loss: 6.7408 - accuracy: 0.0610 - val_loss: 6.8265 - val_accuracy: 0.0688
Epoch 5/50
139623/139623 [==============================] - 33s 240us/step - loss: 6.6437 - accuracy: 0.0699 - val_loss: 6.7638 - val_accuracy: 0.0752
Epoch 6/50
139623/139623 [==============================] - 33s 237us/step - loss: 6.5511 - accuracy: 0.0771 - val_loss: 6.7014 - val_accuracy: 0.0818
Epoch 7/50
139623/139623 [=================

### Short Text

In [9]:
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=20)

There are 15491 unique words.
There are 211492 sequences.


In [10]:
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 5692 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:175: RuntimeWarning: invalid value encountered in true_divide
  embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.06223089,  0.03835242,  0.08488411, ..., -0.04284497,
         0.08662399, -0.00527513],
       ...,
       [-0.00346489, -0.08874972, -0.20891733, ...,  0.09497839,
        -0.12697276, -0.08340222],
       [-0.02349926, -0.06641477, -0.01852101, ...,  0.07821092,
         0.12694902,  0.1010958 ],
       [ 0.02721107,  0.00886596,  0.24643607, ...,  0.02826212,
         0.23311737,  0.06475616]])

#### Bidirectional, 2 layers, Trainable

In [11]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=2,
    bi_direc=True
)

In [12]:
history = utils.train_model(
    training_dict,
    f'{content_type}_bi-2_layer-trainable-20_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 148044 samples, validate on 63448 samples
Epoch 1/50
148044/148044 [==============================] - 25s 171us/step - loss: 7.6587 - accuracy: 0.0365 - val_loss: 6.9924 - val_accuracy: 0.0453
Epoch 2/50
148044/148044 [==============================] - 23s 153us/step - loss: 6.9763 - accuracy: 0.0437 - val_loss: 7.0038 - val_accuracy: 0.0445
Epoch 3/50
148044/148044 [==============================] - 23s 154us/step - loss: 6.9281 - accuracy: 0.0443 - val_loss: 6.9272 - val_accuracy: 0.0484
Epoch 4/50
148044/148044 [==============================] - 23s 154us/step - loss: 6.8086 - accuracy: 0.0531 - val_loss: 6.8809 - val_accuracy: 0.0675
Epoch 5/50
148044/148044 [==============================] - 23s 154us/step - loss: 6.7147 - accuracy: 0.0663 - val_loss: 6.8137 - val_accuracy: 0.0731
Epoch 6/50
148044/148044 [==============================] - 22s 151us/step - loss: 6.6291 - accuracy: 0.0723 - val_loss: 6.7644 - val_accuracy: 0.0837
Epoch 7/50
148044/148044 [=================

#### Unidirectional, 1 layer, Trainable

In [13]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False
)

In [15]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-20_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 148044 samples, validate on 63448 samples
Epoch 1/50
148044/148044 [==============================] - 16s 107us/step - loss: 7.7753 - accuracy: 0.0337 - val_loss: 6.9823 - val_accuracy: 0.0445
Epoch 2/50
148044/148044 [==============================] - 15s 103us/step - loss: 6.9177 - accuracy: 0.0435 - val_loss: 6.9020 - val_accuracy: 0.0445
Epoch 3/50
148044/148044 [==============================] - 15s 104us/step - loss: 6.8180 - accuracy: 0.0454 - val_loss: 6.8894 - val_accuracy: 0.0497
Epoch 4/50
148044/148044 [==============================] - 15s 104us/step - loss: 6.7570 - accuracy: 0.0596 - val_loss: 6.8488 - val_accuracy: 0.0708
Epoch 5/50
148044/148044 [==============================] - 15s 104us/step - loss: 6.6556 - accuracy: 0.0700 - val_loss: 6.7476 - val_accuracy: 0.0751
Epoch 6/50
148044/148044 [==============================] - 15s 104us/step - loss: 6.5429 - accuracy: 0.0796 - val_loss: 6.6742 - val_accuracy: 0.0858
Epoch 7/50
148044/148044 [=================

## Compare Model

<tr>
    <th>Name of Model</th>
    <th>LTSM Layers</th>
    <th>Bidirectional</th>
    <th>Trainable Embeddings</th>
    <th>Sequence Length</th>
    <th>Val Loss</th>
    <th>Val Accuracy</th>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>False</td>
    <td>50</td>
    <td>6.4511</td>
    <td>0.1130</td>
</tr>
<tr>
    <td></td>
    <td>2</td>
    <td>False</td>
    <td>False</td>
    <td>50</td>
    <td>6.5666</td>
    <td>0.1039</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>True</td>
    <td>False</td>
    <td>50</td>
    <td>6.3906</td>
    <td>0.1154</td>
</tr>
<tr>
    <td></td>
    <td>2</td>
    <td>True</td>
    <td>False</td>
    <td>50</td>
    <td>6.5250</td>
    <td>0.1065</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>6.6767</td>
    <td>0.1770</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>True</td>
    <td>True</td>
    <td>50</td>
    <td>6.9115</td>
    <td>0.1669</td>
</tr>
<tr>
    <td></td>
    <td>2</td>
    <td>True</td>
    <td>True</td>
    <td>20</td>
    <td>6.8365</td>
    <td>0.1451</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>6.6482</td>
    <td>0.1757</td>
</tr>

# Sample Output

## Load Model

In [26]:
from keras.models import load_model
# Get Model Weights and Architecture
# MODELS_DIR = os.path.join(os.path.dirname(os.path.abspath('')), 'models')
MODELS_DIR = '/content/drive/My Drive/Code/autocomplete_me/models'
model_filepath = os.path.join(MODELS_DIR, f'{content_type}_uni-1_layer-trainable-20_seq.h5')

model = load_model(model_filepath)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [27]:
# Get Text Data
# text = reader.read_bbc_tech()

TRAINING_LENGTH = 50
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=TRAINING_LENGTH)

There are 15491 unique words.
There are 199462 sequences.


In [28]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=50,
    diversity=1,
    n_gen=1
)

In [29]:
' '.join(word for word in original_sequence)

"own name . Part of the surge in people signing up was due to BT stretching the reach of ADSL - the UK's most widely used way of getting broadband - beyond six kilometres . Asymmetric Digital Subscriber Line technology lets ordinary copper phone lines support high data speeds ."

In [30]:
' '.join(word for word in gen_list[0])

'< --- > internet Bysshe are left recently said , says . All unsolicited attacks offered in data and very . The move is open on universal successfully growing announced for this cell than a premium cards . We means only different software , which are then pay in music . Blogs ,'

In [33]:
' '.join(word for word in a)

'< --- > The standard speed is 512kbps , though faster connections are available . According to BT , more than 95 of UK homes and businesses can receive broadband over the phone line . It aims to extend this figure to 99.4 by next summer . There are also an estimated 1.7'

In [ ]:
# Custom Sentence

In [34]:
sentence = 'Stocks of major large technology firms are becoming'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 5, 524, 490, 47, 130, 16, 503]
conference . The Gracenote service can the firm Centre charge of others , being 92.9 of dedicated online trialled changing file-swapping , says months . He has also seen more granted so face the part of early approach and the Entropia Athlon accounts director of reconditioned graphics part of the


In [37]:
sentence = 'However, there have been many instances of'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 96, 21, 45, 81, 8389, 5]
the huge success . Santy starts 38mph Love , megapixel media could be committed to adhere to learn the services to have quickly . It ever looked , right to be like online with recent difference and wi-fi card on goods directly on their customers and killed inside home different
